# Introduction to Cryptography for Web3 Engineers

## Notebook Kernel Setup

The below enables optional features we want for speed & reuse of build artifacts, should be run at kernel startup, and all dependencies that we need to build _before_ anything else in this notebook will work.

Instead of rebuilding see the `Reset, keeping build artifacts` section to clear REPL state only. 


In [2]:
:sccache 1
:linker lld

:dep sp-core = { version = "6.0.0", git = "https://github.com/paritytech/substrate.git", branch = "polkadot-v0.9.25" }

// Loading & building dependencies crates here takes *a while*! 
// Run this while you move on to the readings below.
// NOTE: A kernel restart removes all target artifacts except those in sccache!
// ONLY restart only if explicitly needed.

sccache: true
linker: lld


## Digital Signatures

Here we demonstrate a few parts of [Substrate's primitive core (`sp_core`)](https://paritytech.github.io/substrate/master/sp_core/index.html) library's component for interacting with keys and signatures.

In [3]:
use sp_core::{
	blake2_256,
	crypto::Derive,
	DeriveJunction,
	Pair as _,
	sr25519::{Pair, Public},
};

## Key generation

In [4]:
// Generate a secret key.
let (pair, mnemonic, raw_seed) = Pair::generate_with_phrase(None);
println!("Secret Phrase: {}", mnemonic);
println!("Secret Key: {:?}", raw_seed);

Secret Phrase: sock limb mammal spirit lion early perfect vacant donkey web galaxy frost
Secret Key: [229, 179, 222, 104, 244, 124, 166, 124, 195, 22, 85, 173, 165, 74, 126, 244, 198, 34, 145, 75, 120, 6, 231, 14, 248, 225, 105, 199, 34, 174, 239, 198]


In [5]:
// Derive the public key.
let pk = pair.public();
println!("Public Key: {:?}\n", pk.0);

Public Key: [162, 182, 134, 110, 56, 211, 154, 186, 153, 58, 155, 137, 211, 195, 120, 102, 80, 253, 4, 117, 49, 254, 29, 60, 121, 1, 149, 157, 225, 133, 171, 126]



## Signatures

In [6]:
// Sign a message.
let message = b"Welcome to Polkadot Blockchain Academy 2022";
let signature = pair.sign(&message[..]);
println!("Message: {:?}", std::str::from_utf8(&message[..]).unwrap());
println!("Signature: {:?}", &signature);

Message: "Welcome to Polkadot Blockchain Academy 2022"
Signature: 7c76df84d34cedd97844e83318d6025d2125579a462f48a5b427a5cf6ad1343c0c8a97f00bd765f96a6db5fe585b913a4d065f79897826f249ce704459009d86


In [7]:
// Verify the signature.
assert!(Pair::verify(&signature, &message[..], &pk));
println!("Signature verified!\n");

Signature verified!



In [8]:
// Alter the message.
let tampered = b"Welcome to Polkadot Blockchain Academy 2021";
assert!(!Pair::verify(&signature, &tampered[..], &pk));
println!("Tampered Message: {:?}", std::str::from_utf8(&tampered[..]).unwrap());
println!("Signature rejected!\n");

Tampered Message: "Welcome to Polkadot Blockchain Academy 2021"
Signature rejected!



In [9]:
/* Message Hash */
let long_message =
	b"Welcome to Polkadot Blockchain Academy 2022. We are staying in Cambridge, which I was told is far superior to Oxford, but I should probably leave that to others to hash out.";
let message_hash = blake2_256(&long_message[..]);
let signature_on_hash = pair.sign(&message_hash);
println!("Longer Message: {:?}", std::str::from_utf8(&long_message[..]).unwrap());
println!("Long Message Hash: {:?}", message_hash);

Longer Message: "Welcome to Polkadot Blockchain Academy 2022. We are staying in Cambridge, which I was told is far superior to Oxford, but I should probably leave that to others to hash out."
Long Message Hash: [32, 248, 188, 171, 200, 39, 71, 8, 146, 106, 11, 83, 138, 3, 249, 154, 134, 149, 104, 188, 38, 250, 1, 130, 161, 99, 253, 184, 125, 137, 253, 116]


In [10]:
// Verify the signature.
assert!(Pair::verify(&signature_on_hash, blake2_256(&long_message[..]), &pk));
println!("Signature verified!\n");

Signature verified!



## Hard Derivation

In [11]:
// Derive new key pairs using //polkadot and //kusama.
let pair_polkadot = Pair::from_string(&format!("{}//polkadot", &mnemonic), None);
let pk_polkadot = pair_polkadot.unwrap().public();
let pair_kusama = Pair::from_string(&format!("{}//kusama", &mnemonic), None);
let pk_kusama = pair_kusama.unwrap().public();
println!("Polkadot Public Key: {:?}", &pk_polkadot.0);
println!("Kusama Public Key: {:?}\n", pk_kusama.0);

Polkadot Public Key: [178, 202, 88, 110, 165, 122, 4, 199, 103, 231, 119, 225, 158, 45, 9, 39, 76, 11, 155, 101, 204, 3, 219, 55, 59, 21, 123, 49, 248, 242, 205, 100]
Kusama Public Key: [208, 228, 186, 120, 37, 135, 46, 177, 103, 116, 119, 250, 186, 49, 195, 158, 99, 187, 218, 176, 34, 133, 119, 64, 217, 253, 188, 39, 120, 27, 138, 81]



## Soft Derivation

In [12]:
// Derive a soft path on the Polkadot key.
let pair_polkadot_zero = Pair::from_string(&format!("{}//polkadot/0", &mnemonic), None);
let pubkey_soft_derived_with_secret = pair_polkadot_zero.unwrap().public();
println!(
	"Polkadot Soft-Derived Public Key (from secret): {:?}",
	&pubkey_soft_derived_with_secret.0
);

// Derive a soft path on the Polkadot key, but only use the _public_ material.
let pk_polkadot: Public = Public(pk_polkadot.0);
let path = vec![DeriveJunction::soft(0u8)];
let pubkey_soft_derived_without_secret = pk_polkadot.derive(path.into_iter());
println!(
	"Polkadot Soft-Derived Public Key (from pubkey): {:?}",
	&pubkey_soft_derived_without_secret.unwrap().0
);

assert_eq!(pubkey_soft_derived_with_secret, pubkey_soft_derived_without_secret.unwrap());
println!("They are equal!");

Polkadot Soft-Derived Public Key (from secret): [86, 244, 24, 127, 228, 0, 73, 142, 195, 188, 135, 123, 68, 97, 208, 39, 120, 154, 10, 98, 112, 144, 205, 181, 9, 42, 139, 47, 244, 9, 172, 26]
Polkadot Soft-Derived Public Key (from pubkey): [86, 244, 24, 127, 228, 0, 73, 142, 195, 188, 135, 123, 68, 97, 208, 39, 120, 154, 10, 98, 112, 144, 205, 181, 9, 42, 139, 47, 244, 9, 172, 26]
They are equal!


### Reset, keeping build artifacts

Run the below cell to "reset" the kernel

In [13]:
:clear // Clear all state, **keeping compilation cache**, use this over a kernel restart when possible. You will need to re-run the :deps to have them loaded into state. 
:last_compile_dir // Show where the deps we last built live, in case you want to recover these

"/tmp/.tmph1dO6j"
